In [2]:
# require Python 3.12: using `type` soft keyword

from enum import Enum
from pprint import pprint
from resources import (
    BaseModel,
    CodeableConcept,
    Coding,
    Condition,
    dateType,
    Encounter,
    HumanName,
    Identifier,
    List_,
    Literal_,
    Observation,
    Organization,
    Patient,
    Procedure,
    Reference,
    Questionnaire,
    QuestionnaireResponse,
)


def show_object(obj: BaseModel):
    """Prints obj attributes that are not not"""
    for k, v in obj.__dict__.items():
        if v:
            pprint(f"{k}: {getattr(obj, k)}")

In [3]:
# resources are FHIR R4B resources, with all fields are optional
patient_base = Patient()
patient_base.__dict__

{'resourceType': 'Patient',
 'id': None,
 'id__ext': None,
 'meta': None,
 'text': None,
 'name': None,
 'link': None,
 'photo': None,
 'active': None,
 'active__ext': None,
 'gender': None,
 'gender__ext': None,
 'telecom': None,
 'address': None,
 'contact': None,
 'language': None,
 'language__ext': None,
 'contained': None,
 'extension': None,
 'birthDate': None,
 'birthDate__ext': None,
 'identifier': None,
 'deceasedBoolean': None,
 'deceasedBoolean__ext': None,
 'deceasedDateTime': None,
 'deceasedDateTime__ext': None,
 'implicitRules': None,
 'implicitRules__ext': None,
 'maritalStatus': None,
 'communication': None,
 'multipleBirthBoolean': None,
 'multipleBirthBoolean__ext': None,
 'multipleBirthInteger': None,
 'multipleBirthInteger__ext': None,
 'modifierExtension': None,
 'generalPractitioner': None,
 'managingOrganization': None}

In [4]:
# IPS has put constrained on Patient resource, some fields are now mandatory
class PatientIPS(Patient):
    name: List_[HumanName] = None
    birthdate: dateType = None


# note given name is a list, and there is only one family name
show_object(
    PatientIPS(
        name=[HumanName(given=["Daniel"], family="Kapitan")], birthdate="09-09-1973"
    )
)

'resourceType: Patient'
('name: [HumanName(id=None, id__ext=None, use=None, use__ext=None, text=None, '
 "text__ext=None, given=['Daniel'], given__ext=None, family='Kapitan', "
 'family__ext=None, prefix=None, prefix__ext=None, suffix=None, '
 'suffix__ext=None, period=None, extension=None)]')
'birthdate: 09-09-1973'


In [5]:
codeEDD = CodeableConcept(
    coding=[
        Coding(
            code="11778-8",
            system="https://loinc.org",
            display="Delivery date Estimated",
        )
    ]
)

In [6]:
Observation(
    status="registered",
    code=codeEDD,
    subject=Reference(
        reference="https://some.fhir/server/dkapitan",
    ),
)

Observation(resourceType='Observation', id=None, id__ext=None, meta=None, text=None, note=None, focus=None, partOf=None, status='registered', status__ext=None, issued=None, issued__ext=None, method=None, device=None, basedOn=None, subject=Reference(id=None, id__ext=None, type=None, type__ext=None, display=None, display__ext=None, extension=None, reference='https://some.fhir/server/dkapitan', reference__ext=None, identifier=None), language=None, language__ext=None, category=None, valueQuantity=None, valueCodeableConcept=None, valueString=None, valueString__ext=None, valueBoolean=None, valueBoolean__ext=None, valueInteger=None, valueInteger__ext=None, valueRange=None, valueRatio=None, valueSampledData=None, valueTime=None, valueTime__ext=None, valueDateTime=None, valueDateTime__ext=None, valuePeriod=None, bodySite=None, specimen=None, contained=None, extension=None, encounter=None, performer=None, hasMember=None, component=None, identifier=None, derivedFrom=None, effectiveDateTime=None, 

In [7]:
# this is a bit of a roundabout way to put constraints on the allowed coding
# but it serves to demonstrate the mechanism
class CodingEdd(Enum):
    _11778_8 = Coding(
        code="11778-8", system="https://loinc.org", display="Delivery date Estimated"
    )
    _11778_6 = Coding(
        code="11778-6",
        system="https://loinc.org",
        display="Delivery date Estimated from last menstrual period",
    )
    _11778_4 = Coding(
        code="11778-4",
        system="https://loinc.org",
        display="Delivery date Estimated from ovulation date",
    )


class CodeableConceptEDD(CodeableConcept):
    coding: List_[Literal_[CodingEdd._11778_8, CodingEdd._11778_6, CodingEdd._11778_4]]

In [8]:
# Observation
# fixed code list how EDD is estimated using LOINC codes
# https://build.fhir.org/ig/HL7/fhir-ips/ValueSet-edd-method-uv-ips.html
# note that ObservationPregnancyEddIPS removes bodysite, method, specimen, device, referenceRange and component
# we haven't implemented that here
# Main point that we demonstrate is value binding
class ObservationPregnancyEddIPS(Observation):
    code: CodeableConceptEDD
    subject: Reference


show_object(
    ObservationPregnancyEddIPS(
        status="registered",
        code=CodeableConceptEDD(coding=[CodingEdd._11778_8]),
        subject=Reference(
            reference="https://some.fhir/server/dkapitan",
        ),
    )
)

'resourceType: Observation'
'status: registered'
('subject: id=None id__ext=None type=None type__ext=None display=None '
 'display__ext=None extension=None '
 "reference='https://some.fhir/server/dkapitan' reference__ext=None "
 'identifier=None')
('code: id=None id__ext=None text=None text__ext=None '
 'coding=[<CodingEdd._11778_8: Coding(id=None, id__ext=None, '
 "system='https://loinc.org', system__ext=None, version=None, "
 "version__ext=None, display='Delivery date Estimated', display__ext=None, "
 "extension=None, userSelected=None, userSelected__ext=None, code='11778-8', "
 'code__ext=None)>] extension=None')
